import libraries

In [ ]:
from besos import eppy_funcs as ef
from besos.eplus_funcs import print_available_outputs
from besos.evaluator import EvaluatorEP
from besos.objectives import MeterReader, clear_outputs
from besos.optimizer import NSGAII
from besos.parameters import FieldSelector, Parameter, RangeParameter, wwr
from besos.problem import EPProblem, Problem

# Objectives and Constraints

`Evaluators` support two types of outputs: Objectives and Constraints.
These are both made using the `MeterReader` and `VariableReader` classes.
The only difference is how they are used by the problem.

First we load the EnergyPlus example file, clear any output data and define some parameters.

In [ ]:
building = ef.get_building()
clear_outputs(building)
inputs = [
    wwr(),
    Parameter(
        FieldSelector(
            class_name="Material",
            object_name="Mass NonRes Wall Insulation",
            field_name="Thickness",
        ),
        RangeParameter(0.01, 0.99),
    ),
]

Objectives and constraints can be specified in various ways.
+ The most explicit is by calling the relevant constructor.

In [ ]:
objectives = [
    MeterReader(
        key_name="Electricity:Facility", class_name="Output:Meter", frequency="Hourly"
    )
]
EPProblem(outputs=objectives)

+ The most concise is a list of the `key_names`.

The constructor has defaults, so we can often omit `class_name` and `frequency`.
A list of key names will be automatically be converted by `EPProblem`.
Meters and variables that do not have a `frequency` specified will default to any frequency that is already used for that output, or if none is used yet then they will use Hourly.

In [ ]:
objectives = ["Electricity:Facility"]
EPProblem(outputs=objectives)

+ Using `Problem`

If we do not need the output-reading features of meters, we can use `Problem` instead of `EPProblem`, and they will be converted to `Objective` objects which act as placeholders.
`EPProblem` converts them to `Meter:Reader` objects.
Either of these conversions can be overriden using the converters argument.

In [ ]:
objectives = ["any", "names", "work"]
Problem(outputs=objectives)

+ Specifying the aggregation function

The `func` argument is used define how to aggregate the individual time series results.
By default, all measurements are summed.
If we wanted to instead minimize the variance, we can write our own aggrgation function.
Here we define two electricity objectives, the first summing the hourly values and the second taking the variance.

In [ ]:
def variance(result):
    return result.data["Value"].var()


objectives = [
    MeterReader("Electricity:Facility", name="Electricity Usage"),
    MeterReader("Electricity:Facility", func=variance, name="Electricity Variance"),
]

When we want to specify the direction of optimisation, we can use `minmize_outputs` (defaults to `true` for all objectives).
Here we say we want to search for a design that has:
+ low electricty use (minimize objective 1 defined above)
+ high variability of electricity use (maximize objective 2 defined above)
+ less than 800 kgCO2 (constraint)

In [ ]:
evaluator = EvaluatorEP(
    EPProblem(
        inputs=inputs,
        outputs=objectives,
        minimize_outputs=[True, True],
        constraints=["CO2:Facility"],
        constraint_bounds=["<=800"],
    ),
    building,
    out_dir="outputdir",
)

In [ ]:
# this cell runs the optimisation
results1 = NSGAII(evaluator, evaluations=1, population_size=10)
results1

In [ ]:
results1.describe()

In [ ]:
# this cell runs the optimisation
results2 = NSGAII(evaluator, evaluations=10, population_size=10)
results2

In [ ]:
results2.describe()

## Get available objectives

The user can use print_available_outputs to print out the available objectives for this building

In [ ]:

building = ef.get_building(mode="idf")
print_available_outputs(building, name="facility", frequency="monthly")